# 1. 지하철역 좌표 기반 법정동 코드 매핑

In [8]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv
import pickle

load_dotenv()

KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

HEADERS = {
    "Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"
}

COORD2REGION_URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"

# env 정상인지 확인
print(KAKAO_REST_API_KEY)


8d5e873d4ae52b36838aef3796e62a8e


In [9]:
INPUT_PATH = "/Users/choiwoojin/2026/rentsignal-data-collector/raw/subway_info.csv"

df = pd.read_csv(INPUT_PATH, encoding="euc-kr")

# 좌표 숫자 변환
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df = df.dropna(subset=["경도", "위도"])

# coord_key 생성
df["coord_key"] = df["경도"].astype(str) + "_" + df["위도"].astype(str)

# 고유 좌표 추출
coords = df[["경도", "위도"]].drop_duplicates().reset_index(drop=True)
coords["coord_key"] = coords["경도"].astype(str) + "_" + coords["위도"].astype(str)

print("총 고유 좌표 수:", len(coords))

총 고유 좌표 수: 780


In [10]:
coord_map = {}
OUTPUT_PATH = "/Users/choiwoojin/2026/rentsignal-data-collector/raw/subway_join_legaldong.csv"
for idx, row in coords.iterrows():

    params = {
        "x": row["경도"],
        "y": row["위도"]
    }

    try:
        res = requests.get(
            COORD2REGION_URL,
            headers=HEADERS,
            params=params
        )

        if res.status_code == 429:
            print("Rate limit 발생 → 1초 대기")
            time.sleep(1)
            continue

        if res.status_code != 200:
            print("HTTP 에러:", res.status_code)
            coord_map[row["coord_key"]] = (None, None)
            continue

        data = res.json()

        code = None
        name = None

        for doc in data.get("documents", []):
            if doc.get("region_type") == "B":
                code = doc.get("code")
                name = doc.get("region_3depth_name")
                break

        coord_map[row["coord_key"]] = (code, name)

        # rate limit 보호
        time.sleep(0.1)

        # 진행 로그
        if idx % 500 == 0:
            print(f"진행률: {idx}/{len(coords)}")

        # 중간 저장
        if idx % 1000 == 0 and idx != 0:
            with open("coord_map_backup.pkl", "wb") as f:
                pickle.dump(coord_map, f)
            print("중간 저장 완료")

    except Exception as e:
        print("에러 발생:", e)
        coord_map[row["coord_key"]] = (None, None)

print("API 매핑 완료")


df["법정동코드"] = df["coord_key"].map(lambda k: coord_map.get(k, (None, None))[0])
df["법정동명"] = df["coord_key"].map(lambda k: coord_map.get(k, (None, None))[1])

# 임시 key 제거
df = df.drop(columns=["coord_key"])


df.to_csv(
    OUTPUT_PATH,
    index=False,
    encoding="utf-8"
)

print("저장 완료:", OUTPUT_PATH)

진행률: 0/780
진행률: 500/780
API 매핑 완료
저장 완료: /Users/choiwoojin/2026/rentsignal-data-collector/raw/subway_join_legaldong.csv
